# Statistiques Appliquées

Python 3.5 needed

## Factorization of matrices

### Matrix Factorization (MF)


In [1]:
import numpy as np

#Creating the new matrix
def MF(A,d=15,steps=100,alpha=0.0001,beta=0.0001,threshold=0.01):
    n,m = A.shape
    P=np.random.rand(n,d)
    Q=np.random.rand(d,m)
    for step in range(steps):
        P += alpha * (2*(A @ Q.T - P @ Q @ Q.T) - beta * P)
        Q += alpha * (2*(P.T @ A - P.T @ P @ Q) - beta * Q)
        PQ = P @ Q
        euclidean = np.nansum((A-PQ)**2)
        if euclidean < threshold:
            break
    print("The euclidean distance is {d}".format(d=euclidean))
    return(PQ, P, Q)

Let's apply it on a matrix of 1s and 5s, with perturbation.

In [2]:
# Dimension of latent space
d=15

nbUsers = 943
nbFilms = 1682

# Create matrices
def simulateData(d=15, sigma=0, nbRows = 943, nbCols = 1682):
    newData = np.ones((nbRows,nbCols))
    
    
    rowSplits = np.sort(np.random.choice(range(1,nbRows), d-1, replace=False))
    rowStarts = np.concatenate([np.array([0]),rowSplits]) 
     
    colSplits = np.sort(np.random.choice(range(1,nbCols), d-1, replace=False))
    colStarts = np.concatenate([np.array([0]),colSplits])
    
    for i in range(1,d):
        fives = np.zeros((rowStarts[i]-rowStarts[i-1],colStarts[i]-colStarts[i-1]))+5
        newData[rowStarts[i-1]:rowStarts[i],colStarts[i-1]:colStarts[i]] = fives
    
    fives = np.zeros((nbRows-rowStarts[d-1],nbCols-colStarts[d-1]))+5
    newData[rowStarts[d-1]:,colStarts[d-1]:] = fives
    
    
    newData += sigma * (np.random.randn(nbRows,nbCols)-1/2)
    return newData

In [3]:
A = simulateData(15,0.05,943,1682)
MF(A,15)

The euclidean distance is 126398.80419450438


(array([[ 0.97180892,  0.99498776,  1.1676196 , ...,  1.12670146,
          0.89410988,  1.12613139],
        [ 1.08812725,  1.04660435,  1.18471553, ...,  1.07965429,
          0.95734531,  1.11357196],
        [ 1.05089552,  0.98252975,  1.1014586 , ...,  1.05972373,
          0.92200987,  1.13767892],
        ..., 
        [ 1.24245738,  1.44160698,  1.18022623, ...,  4.34233762,
          4.4781173 ,  4.2499739 ],
        [ 1.25508738,  1.44777677,  1.18170276, ...,  4.37136214,
          4.46433653,  4.25700511],
        [ 1.22178466,  1.48267549,  1.13399294, ...,  4.42132438,
          4.4377212 ,  4.28203156]]),
 array([[  1.88034665e-01,  -1.81994604e-02,  -1.91208504e-02, ...,
           8.03363735e-02,   3.26088633e-01,  -1.95685459e-02],
        [  1.26785453e-01,   1.86593265e-04,  -1.85543703e-01, ...,
           1.68447659e-01,   2.70097554e-01,  -2.41648408e-03],
        [  9.04116705e-02,  -1.69095555e-03,  -3.33906640e-02, ...,
           1.35055756e-01,   2.97915787e

### Non-Negative Matrix Factorization (NMF)

Refers to the article : http://papers.nips.cc/paper/1861-algorithms-for-non-negative-matrix-factorization.pdf

The problem is defined with two distances.

Sklearn version

In [4]:
from sklearn.decomposition import NMF

#Calculate predicted ratings with NMF, sklearn version
def NMFsk(A,d=15):
    A=np.array(A)
    nmf = NMF(n_components=d)
    P = nmf.fit_transform(A);
    Q = nmf.components_;
    euclidean = np.nansum((A-P @ Q)**2)
    print("The euclidean distance is {d}".format(d=euclidean))
    return (P @ Q, P, Q)

C:\Users\lamjo_000\Anaconda3\envs\python3\lib\site-packages\sklearn\utils\fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


In [5]:
NMFsk(A,15)

The euclidean distance is 3863.5191437865


(array([[ 4.98710422,  4.99700359,  4.9799231 , ...,  0.96844825,
          0.96433159,  0.97703689],
        [ 4.96078961,  4.97063063,  4.95364629, ...,  0.97349256,
          0.969329  ,  0.98210653],
        [ 4.9916107 ,  5.00157827,  4.98451541, ...,  0.96776042,
          0.9636541 ,  0.97650984],
        ..., 
        [ 0.97326314,  0.97934396,  0.97095945, ...,  4.99118932,
          4.96805857,  4.99666699],
        [ 0.94419717,  0.95020161,  0.94202404, ...,  4.99349694,
          4.97033802,  4.99866406],
        [ 0.98166033,  0.98782012,  0.97934669, ...,  4.99561073,
          4.97250463,  5.00066251]]),
 array([[  3.21480623e-01,   8.86030454e-02,   1.14116996e-01, ...,
           1.22270933e-01,   1.82399519e-02,   1.81582871e+00],
        [  3.19851605e-01,   9.47086503e-02,   1.15329052e-01, ...,
           1.26606564e-01,   1.67598404e-02,   1.80503008e+00],
        [  3.20502709e-01,   9.20873469e-02,   1.21737178e-01, ...,
           1.23775047e-01,   1.23805131e

Kullback-Leibler

In [6]:
#Updates the matrices P and Q for an iteration
def update(A, P, Q, PQ, A_over_PQ):
    # equation (5)
    Q *= (A_over_PQ.T @ P / P.sum(axis=0)).T
    PQ = P @ Q
    A_over_PQ = A / PQ
    P *= (A_over_PQ @ Q.T) / Q.sum(axis=1)

    PQ = P @ Q
    P_over_PQ = A / PQ
    return P, Q, PQ, A_over_PQ


def NMFdiv(A, d=15, iterations=100):
    n, m = A.shape
    P = np.random.random(n * d).reshape(n, d) * 4 + 1
    Q = np.random.random(d * m).reshape(d, m) * 4 + 1
    PQ = P @ Q
    A_over_PQ = A / PQ
    for i in range(iterations):
        P, Q, PQ, A_over_PQ = update(A, P, Q, PQ, A_over_PQ)
        # equation (3)
        divergence = np.nansum((A * np.log(A_over_PQ)) - A + PQ)
    print("The Kullback–Leibler divergence is {d}".format(d=divergence))
    return(P @ Q, P, Q)

In [7]:
NMFdiv(A,15,101)

The Kullback–Leibler divergence is 73913.48744363716


(array([[ 1.67882058,  1.65813213,  1.66926519, ...,  0.97066025,
          0.96952884,  0.97039909],
        [ 1.67330389,  1.66986557,  1.66482246, ...,  0.97629309,
          0.98995033,  0.97195157],
        [ 1.69297407,  1.67110203,  1.68002244, ...,  0.97156291,
          0.96980872,  0.9687501 ],
        ..., 
        [ 0.92854273,  0.93050316,  0.92376562, ...,  5.00414241,
          4.98994518,  4.98654408],
        [ 0.95063742,  0.94836182,  0.94463824, ...,  5.00965591,
          4.99468908,  4.99151533],
        [ 0.91812533,  0.93301208,  0.91054441, ...,  5.0010559 ,
          4.99577333,  4.97748021]]),
 array([[  2.17872723e+00,   2.32525161e+00,   2.66910091e+00, ...,
           1.28370873e+00,   4.03535124e+00,   9.10634129e-01],
        [  2.12813424e+00,   2.44916128e+00,   2.75446933e+00, ...,
           1.07615718e+00,   3.07314941e+00,   7.39383782e-01],
        [  2.15121038e+00,   2.41391154e+00,   2.68271689e+00, ...,
           1.07957434e+00,   4.13384074e

Euclidean

In [8]:
def update(A, P, Q):
    # equation (6)
    Q *= P.T @ A / (P.T @ P @ Q)
    P *= (A @ Q.T) / (P @ Q @ Q.T)
    return P, Q


def NMFeuc(A, d, iterations=100):
    n, m = A.shape
    P = np.random.random(n * d).reshape(n, d) * 4 + 1
    Q = np.random.random(d * m).reshape(d, m) * 4 + 1
    for i in range(iterations):
        P, Q = update(A, P, Q)
        # equation (4)
        PQ = P @ Q
        euclidean = np.nansum((A-PQ)**2)
    print("At iteration {i}, the euclidean distance is {d}".format(i=i, d=euclidean))
    return(P @ Q, P, Q)

In [9]:
NMFeuc(A,15)

At iteration 99, the euclidean distance is 26776.17872483564


(array([[ 1.07905251,  1.07739115,  1.07634101, ...,  0.98442668,
          0.97913325,  0.98336137],
        [ 1.12579474,  1.1272693 ,  1.12233953, ...,  0.99333828,
          0.98417089,  0.98964348],
        [ 1.13075565,  1.13184701,  1.12530959, ...,  0.9879241 ,
          0.97897018,  0.98597305],
        ..., 
        [ 1.1680637 ,  1.17456047,  1.16366485, ...,  4.99398119,
          4.96591452,  4.98522202],
        [ 1.16899827,  1.1762989 ,  1.16521949, ...,  4.99500436,
          4.96708038,  4.98603153],
        [ 1.16418785,  1.17032823,  1.15963821, ...,  5.00143165,
          4.97327617,  4.99270848]]),
 array([[  2.82675219e+00,   8.88332494e-01,   2.17966194e+00, ...,
           2.15114520e+00,   1.56733910e+00,   3.58178710e+00],
        [  2.77765726e+00,   6.26690503e-01,   2.36940755e+00, ...,
           2.14702507e+00,   1.53334372e+00,   3.24359226e+00],
        [  2.65029813e+00,   4.19782749e-01,   2.76947723e+00, ...,
           2.14646573e+00,   1.26824887e

## Data from MovieLens

In [10]:
# Loading dataframes

import pandas
dfDirectory = "C:\\Users\\lamjo_000\\Google Drive\\Stat App\\Data\\"

dataframeTrain = pandas.read_csv(dfDirectory + "u1base.base", sep="\t")
dataframeTest = pandas.read_csv(dfDirectory + "u1test.test", sep="\t")

#Shape of the dataframe :
#for each user (column 0)
#   for each film (column 1)
#        rating of the film according to the user (column 2)
dataframeTrain.head()

,ID_User,ID_Movie,Rating,Time
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [11]:
nbUsers = 943
nbFilms = 1682

#Creating the corresponding matrix
def dfToArray(dataframe, nbUsers = 943, nbFilms = 1682):
    # Deleting variable time if it has not been deleted yet.
    try:
        del dataframe["Time"]
    except:
        pass
    ratings = np.zeros((nbUsers,nbFilms))
    for i in range(dataframe.shape[0]):
        #ratings[user][film] = rating
        #indices of users and films start from 1, not from 0.
        ratings[dataframe.ix[i][0]-1][dataframe.ix[i][1]-1] = int(dataframe.ix[i][2])
    return(ratings)

#Dictionary with key: matrix line; content: marked film id
#{user : [films rated by user]}
def ratedFilms(ratings):
    ratedDict = {}
    for i in range(ratings.shape[0]):
        ratedDict[i]=[]
        for k in range(ratings.shape[1]):
            if ratings[i][k] != 0:
                ratedDict[i].append(k)
    return(ratedDict)

# Dictionary with predicted marks
def roundDown(x):
    if x < 1.5:
        return(1)
    elif x >= 4.5:
        return(5)
    else:
        return(round(x))

#Keep predicted values for (user,film) where the couple exists in the test dataframe.
def roundRatings(ratedTest,ratings,roundIt):
    if roundIt:
        applyFct = roundDown
    else:
        applyFct = lambda x : x
    roundedRatings = {}
    for user in ratedTest:
        for film in ratedTest[user]:
            roundedRatings[(user,film)] =  applyFct(ratings[user][film])
    return roundedRatings

In [12]:
# Create arrays
ratingsTrain = dfToArray(dataframeTrain)
ratingsTest = dfToArray(dataframeTest)        

#We pull out every empty column in Train.
#Here is a list of columns with only 0s
#Reverse, so that the index isn't changed during the process of deleting the columns (start from the end)
matrix_sum = reversed(np.where(np.nansum(ratingsTrain,axis=0) == 0)[0])
        
for i in matrix_sum:
    ratingsTrain = np.delete(ratingsTrain, i, 1)
    ratingsTest = np.delete(ratingsTest, i, 1) #We don't want to lose the indices in test.
            
ratedTest = ratedFilms(ratingsTest)
        
# Create dictionary with real ratings
ratingsTestDic = roundRatings(ratedTest, ratingsTest, False)

For data with missing values, Matrix factorization is not enough. We need Matrix Completion.

## Matrix Completion

Check accuracy of predicted values

In [13]:
#Comparison with test set
def calculateAccuracy(roundedRatings,ratingsTestDic):
    errorMax = 4 #Maximum distance to true value is 4.
    error = 0
    for coupleUF in roundedRatings:
        error += abs(roundedRatings[coupleUF] - ratingsTestDic[coupleUF])
    return 1 - error/(errorMax*len(ratingsTestDic))

### Average, that is naive prediction

We change missing data to the average per user. This can be used as a naive prediction of the missing data, and thus as a benchmark for the accuracy of the prediction.

In [14]:
#Mean per user and corresponding dictionary
#NMF averaged
def averageUser(ratings):
    rowMean = np.average(ratings, axis=1, weights = ratings.astype(bool))
    return(rowMean)

#Creating matrix, replacing 0s with means
def replace0s(ratings,rowMean):
    noZeroRatings = ratings.copy()
    inds = np.where(ratings == 0)
    noZeroRatings[inds]=np.take(rowMean,inds[0])
    return(noZeroRatings)

In [15]:
# Create list of average ratings by user
rowMean = averageUser(ratingsTrain)
ratings = replace0s(ratingsTrain, rowMean)

# Create dictionary with predicted ratings
ratings = roundRatings(ratedTest,ratings,True)


# Calculate accuracy
print(calculateAccuracy(ratings,ratingsTestDic))

0.793820112179


### Average, then NMF

Another possibility is to apply NMF, once the missing data have been replaced by the average by user.

In [16]:
# Create list of average ratings by user
rowMean = averageUser(ratingsTrain)
ratings = replace0s(ratingsTrain, rowMean)

# Create new matrix with NMF for euclidean distance.
ratings,P,Q = NMFeuc(ratings,9,1000)

# Create dictionary with predited ratings
ratings = roundRatings(ratedTest,ratings,True)

# Calculate accuracy
print(calculateAccuracy(ratings,ratingsTestDic))

At iteration 999, the euclidean distance is 73816.10639638393
0.80462489984


### Weighted Non Negative Matrix Factorization (WNMF)

http://www.siam.org/meetings/sdm06/proceedings/059zhangs2.pdf 3.2

The only formalized version that was found is for the euclidean distance. Thus, starting from now, we won't consider the Kullback–Leibler divergence anymore.

In [17]:
def update(weightedA, P, Q, Weights):
    # equation (5)
    weightedPQ = Weights * (P @ Q)
    P *= (weightedA @ Q.T) / (weightedPQ @ Q.T)
    weightedPQ = Weights * (P @ Q)
    Q *= (P.T @ weightedA) / (P.T @ weightedPQ)
    return P, Q


def WNMF(A, d, iterations=100):
    n, m = A.shape
    P = np.random.random(n * d).reshape(n, d) * 4 + 1
    Q = np.random.random(d * m).reshape(d, m) * 4 + 1
    Weights = A.copy()
    Weights[Weights != 0] = 1
    weightedA = A # =Weights * A 
    for i in range(iterations):
        P, Q = update(weightedA, P, Q, Weights)
        PQ = P @ Q
        weightedPQ = Weights * PQ
        euclidean = np.nansum((weightedA-weightedPQ)**2)
    print("The euclidean distance is {d}".format(d=euclidean))
    return(P @ Q, P, Q)

In [18]:
# Create new matrix with WNMF.
ratings,P,Q = WNMF(ratingsTrain,9,1000)

# Create dictionary with predicted ratings
ratings = roundRatings(ratedTest,ratings,True)

# Calculate accuracy
print(calculateAccuracy(ratings,ratingsTestDic))

The euclidean distance is 39390.840571025525
0.806891025641


### Learning d

In [19]:
import heapq
#Take 2 dictionaries to compare and find the TPR
def calculateTPR(ratings,ratingsTestDic,NBest=5, nbUsers=943):
    TPRlist = []
    for user in range(nbUsers):
        TPRuser = 0
        ratingsUser = {}
        for (userT,film) in ratingsTestDic:
            if userT == user: # select films from test data
                ratingsUser[film] = ratingsTestDic[(user,film)]
        #Take NBest highest ratings of user       
        NBestlist = heapq.nlargest(NBest, ratingsUser, key=ratingsUser.get)
        for best in NBestlist:
            #calculate TPR with a maximum distance of 1 (see report)
            if abs(ratingsTestDic[(user,best)] - roundDown(ratings[(user,best)])) < 2:
                TPRuser += 1
        #If there are enough films in test for the user        
        if len(ratingsUser) >= NBest:
            TPRlist.append(TPRuser)
    TPR = sum(TPRlist)/(NBest*float(len(TPRlist)))
    #return average TPR
    return TPR

In [20]:
#Figure 2
mat = [["d","TPR"]]
for d in range(1,101):
    print(d)
    # Create new matrix, applying the WNMF
    ratings,P,Q = WNMF(ratingsTrain,d,1000)

    # Create dictionary with predicted ratings
    # Don't round the ratings now, so that a ranking of the ratings may be possible
    roundedRatings = roundRatings(ratedTest,ratings,False)

    # Calculate de TPR
    mat += [[d,calculateTPR(roundedRatings,ratingsTestDic)]]


1
The euclidean distance is 66394.66103688107
2
The euclidean distance is 58886.93423551466
3
The euclidean distance is 54692.842714007675
4
The euclidean distance is 51104.47555297158
5
The euclidean distance is 48199.10356696432
6
The euclidean distance is 45809.24398149295
7
The euclidean distance is 43249.26827791491
8
The euclidean distance is 41330.46161075259
9
The euclidean distance is 39244.71494303611
10
The euclidean distance is 37455.751761842366
11
The euclidean distance is 35716.07306467262
12
The euclidean distance is 34492.998118453856
13
The euclidean distance is 33043.19607312435
14
The euclidean distance is 31465.850770441015
15
The euclidean distance is 30132.773037978288
16
The euclidean distance is 28998.15907659244
17
The euclidean distance is 27647.04054110751
18
The euclidean distance is 26677.155105400645
19
The euclidean distance is 25680.18240875936
20
The euclidean distance is 24836.552412079982
21
The euclidean distance is 23862.890637423847
22
The euclide

## Clusterisation

In [21]:
#Empty 93% of data
#We need to keep at least one non-missing value per row and per column. 
def addMissing(data):
    nbRows, nbCols = data.shape
    #Goal : create an array of 0s and 1s.
    #Then, once we multiply it with our original data elementwise, we only keep data where there was a 1.
    wCleaner = np.zeros((nbRows, nbCols))
    
    #Keep at least a 1 in each row
    l = np.random.choice(range(nbCols), nbRows)
    for i in range(nbRows):
        wCleaner[i][l[i]] = 1
    
    #Keep at least a 1 in each column
    l = np.random.choice(range(nbRows), nbCols)
    for j in range(nbCols):
        wCleaner[l[j]][j] = 1
        
    #Put 1s in random positons    
    l = np.random.choice(range(nbRows*nbCols), int(0.07 * nbRows*nbCols), False)
    for i in range(nbRows):
        for j in range(nbCols):
            if nbCols*i+j in l:
                wCleaner[i][j] = 1
    return (wCleaner * data)

In [22]:
data = addMissing(A)

### K-means

In [23]:
def kMeansApply(steps, nbSamples, nbFeatures, nbClusters, initialCenters, Matrix):#initialCenters = r or centerspp
    centersClusters=np.zeros((nbSamples, nbFeatures))
    centersClusters[:nbClusters,:] = Matrix[initialCenters,:]
    usersClusters={}
    for i in range(nbClusters):
        usersClusters[i]=[]
        
    for j in range(steps):
        # Find closest cluster to user
        for i in range(nbSamples):
            closest=0
            #find minimum for L2
            for k in range(nbClusters):
                if np.linalg.norm(np.array(centersClusters[k])-np.array(Matrix[i])) <= np.linalg.norm(np.array(centersClusters[closest])-np.array(Matrix[i])):
                    closest=k
                    
            #if Matrix[i] (which is an array) not in usersClusters[closest]:
            if not any((Matrix[i] == x).all() for x in usersClusters[closest]):
                usersClusters[closest].append(Matrix[i])  
    
        #Change centers of clusters
        #Reinitialize clusters
        if j != steps-1:    
            for i in range(nbClusters):
                if usersClusters[i] != []:
                    centersClusters[i]=np.sum(usersClusters[i],axis=0)/len(usersClusters[i])
                    usersClusters[i]=[] 
    return (usersClusters,centersClusters)

# Initialization : the centers of the clusters are samples at random.

def kMeans(M, nbClusters, steps):
    Matrix=np.array(M)
    nbSamples, nbFeatures = Matrix.shape
    
    #Initialize centers of clusters
    ##Generate nbClusters random integers in [0,nbSamples) without replacement
    initialCenters = np.random.choice(nbSamples, nbClusters, replace=False)
    usersClusters, centersClusters = kMeansApply(steps, nbSamples, nbFeatures, nbClusters, initialCenters, Matrix)
        
    return (usersClusters, centersClusters)




In [24]:
nbClusters = 15
steps = 100
(usersClusters,centersClusters)=kMeans(A, nbClusters, steps)

### K-means++

In [25]:
# Fill clusters with users thanks to labels.
# Output : centers and indices of users associated with each center
def orderLabels(Matrix, linkUserCluster):
    associatedUsers = {}
    centers = []
    for cluster in linkUserCluster:
        associatedUsers[cluster] = [user for user, clus in enumerate(linkUserCluster) if cluster == clus]
    for cluster in associatedUsers:
        l = []
        for user in associatedUsers[cluster]:
            l.append(Matrix[user])
        centers.append(np.sum(l, axis=0)/len(l))
    return (centers,associatedUsers)

Sklearn version

In [26]:
from sklearn import cluster
#Figure 3
# Number of clusters
nbClusters=30

# Clusterize
kmeansAlg = cluster.KMeans(n_clusters=nbClusters, random_state=0).fit(A)
linkUserCluster = kmeansAlg.predict(A) #list which associates a user to their closest cluster [index of cluster]
print("Clustering done")

(centers,associatedUsers) = orderLabels(ratingsTrain,linkUserCluster)

print(associatedUsers) #{index cluster : [user in cluster]}


Clustering done
{0: [438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476], 1: [205, 207, 218, 223, 231, 253, 259, 263, 270, 273, 276, 326, 332, 346, 347, 374, 388, 391], 2: [23, 25, 30, 35, 43, 44, 45, 46, 51, 52, 54, 55, 59, 63, 65, 67, 69, 71, 72, 75, 76, 77, 79, 83, 85, 87, 88, 90, 91, 92, 93, 107, 109, 111, 113, 114, 117, 123, 125, 127, 132, 138, 139, 140, 144, 146, 147, 148, 151, 152, 153, 158, 159, 160, 164, 166, 167, 171, 173, 178, 180, 181, 185, 186, 188, 190, 195], 3: [520, 523, 533, 537, 538, 539, 540, 553, 557, 561, 565, 566, 572, 586, 588, 589, 591, 597, 599, 603, 606, 608, 613, 615, 622, 623, 624, 626, 630, 631, 632, 633, 644, 645, 653, 655, 658, 659, 665, 666, 670, 671], 4: [781, 782, 785, 789, 806, 810, 813, 816, 820, 835, 844, 846, 856, 863, 874, 878, 886, 903], 5: [675, 676, 678, 679, 680, 681, 682, 685, 686, 687, 689, 692, 693, 

our version

In [27]:
#Figure 3
# Initalize for kmeans++
def kmeansInit(Matrix, nbClusters):
    #nbClusters the number of clusters
    
    check = True
    while check:

        #nbSamples the number of individuals, nbFeatures the number of films
        nbSamples, nbFeatures = Matrix.shape

        #Calculate norm 2 for each user
        usersL2 = np.linalg.norm(Matrix, 2, axis=1)

        #Create matrix of barycenters and list of barycenters
        centers = np.empty((nbClusters, nbFeatures))
        centersID = np.empty(nbClusters,dtype=int)

        #Choose the first barycenter at random
        centerID = np.random.randint(nbSamples)
        centersID[0] = centerID
        centers[0]=Matrix[centerID]

        #Initialize matrix of squared distances
        distSq = np.empty((nbClusters-1,nbSamples))

        #Initialize list of smallest squared distances
        closestDistSq = np.empty(nbSamples)

        #Initialize list of smallest cumulated squared distances
        aggregateClosestDistSq = np.empty(nbSamples)

        #Choose other barycenters
        for iter1 in range(nbClusters-1):
            #Update matrix of distances
            distSq[iter1] = (np.linalg.norm(Matrix - (np.array([centers[iter1]]).T @ np.ones((1,nbSamples))).T, 2, axis=1))**2

            #Update list of smallest distances
            closestDistSq = distSq.min(axis=0)

            #Update list of smallest cumulated distances
            aggregateClosestDistSq[0] = closestDistSq[0]
            for iter2 in range(len(closestDistSq)-1):
                aggregateClosestDistSq[iter2 +1] = closestDistSq[iter2 +1] + aggregateClosestDistSq[iter2]

            #Choose a random real number between 0 and the potential
            #Choose the new center according to a probability proportional to the squared distance between the considered point and the former point.
            rand_nb = np.random.random() * aggregateClosestDistSq[-1]

            #Choose barycenter
            centerID = nbSamples - 1
            while rand_nb < aggregateClosestDistSq[centerID - 1] and centerID > 0:
                centerID -= 1

            #Update centers and centers_id
            centersID[iter1 + 1] = centerID
            centers[iter1 + 1] = Matrix[centerID]

        #In case of probability=0 or the same barycenter is chosen multiple times
        if len(set(centersID)) == nbClusters:
            check = False

    #returns indices (between 0 and 942) of kept barycenters    
    return centersID


#kmeans++
#returns a dictionary with applied partitioning
def kmeansPP(M, nbClusters, steps):
    Matrix=np.array(M)
    nbUsers, nbFilms = Matrix.shape
    initialCenters = kmeansInit(Matrix, nbClusters)
    
    usersClusters, centersClusters = kMeansApply(steps, nbUsers, nbFilms, nbClusters, initialCenters, Matrix)
        
    return (usersClusters,centersClusters)
    

The clustering is not as good as expected when you apply kmeans on data with missing values:

In [28]:
# Number of clusters
nbClusters=15

# Clustering
kmeans = cluster.KMeans(n_clusters=nbClusters, random_state=0).fit(data)
linkUserCluster = kmeans.predict(data)
print("Clustering done")

(centers,associatedUsers) = orderLabels(data,linkUserCluster)

print(associatedUsers)

Clustering done
{0: [125], 1: [480, 481, 676, 677, 678, 679, 680, 681, 683, 684, 686, 687, 688, 689, 690, 691, 693, 695, 696, 697, 699, 701, 702, 703, 704, 705, 706, 708, 709, 710, 713, 714, 715, 716, 717, 718, 720, 721, 723, 724, 725, 726, 728, 729, 733, 734, 735, 736, 737, 739, 741, 742, 743, 744, 747, 749, 750, 751, 752, 754, 755, 756, 757, 758, 927, 932], 2: [71], 3: [30, 32, 33, 35, 38, 39, 43, 45, 46, 47, 51, 52, 60, 62, 63, 65, 69, 73, 75, 85, 90, 92, 93, 97, 98, 107, 108, 111, 112, 115, 116, 119, 120, 126, 127, 134, 136, 149, 150, 157, 159, 161, 170, 171, 172, 174, 177, 179, 182, 185, 190, 196, 197, 199], 4: [31], 5: [147], 6: [928], 7: [19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 34, 37, 41, 42, 44, 48, 49, 50, 53, 54, 55, 56, 57, 58, 59, 61, 64, 66, 68, 70, 72, 74, 76, 77, 78, 79, 80, 81, 82, 83, 84, 86, 87, 88, 89, 91, 94, 95, 96, 99, 100, 101, 102, 103, 104, 105, 106, 109, 110, 113, 114, 117, 118, 121, 122, 123, 124, 128, 129, 130, 131, 132, 133, 135, 137, 138, 140, 141, 142, 1

### NMF, then K-means

If we apply complete the data (using NMF for instance). It is already better.

In [29]:
#Figure 4

#Segmentation on the completed matrix

# Create new matrix with WNMF.
predicted,P,Q = WNMF(data,15,1000)

# Apply clustering
kmeans = cluster.KMeans(n_clusters=nbClusters, random_state=0).fit(predicted)
linkUserCluster = kmeans.predict(predicted)
print("Clustering done")

(centers,associatedUsers) = orderLabels(A,linkUserCluster)

print(associatedUsers)

The euclidean distance is 365.5577611140053
Clustering done
{0: [930, 939], 1: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370,

However, because of the curse of dimensionality, it could be even better if we apply it on the factor matrix of users in order to clusterize them:

In [30]:
#Figure 4

#Segmentation on the completed matrix

# Apply clustering
kmeans = cluster.KMeans(n_clusters=nbClusters, random_state=0).fit(P)
linkUserCluster = kmeans.predict(P)
print("Clustering done")

(centers,associatedUsers) = orderLabels(P,linkUserCluster)

print(associatedUsers)

Clustering done
{0: [19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199], 1: [481], 2: [927, 933, 934, 935, 942], 3: [761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 

## Learning k

In [31]:
#Basic functions to treat error
#Normalize vector
def normalized(a):
    n = np.linalg.norm(a)
    result = []
    for i in range(len(a)):
        result.append(a[i]/float(n))
    return np.array(result)

#Calculate error to learn k
def learningk(Matrix,centers,associatedUsers):
    result = 0
    for cluster in associatedUsers:
        for user in associatedUsers[cluster]:
            result += np.linalg.norm(centers[cluster] - normalized(Matrix[user]))**2
    return result

Learn number of clusters of users and films

In [32]:
# Figure 5 Figure 6

d=15

# Get P and Q
nmf = NMF(n_components=d)
P = nmf.fit_transform(ratingsTrain)
Q = nmf.components_
Q = Q.T

############################
### Clustering on P
############################

#Get error depending on the number of clusters
mat = [["nbClusters","error"]]

for nbClusters in range(1,101):    
    print(nbClusters)

    # Clustering
    kmeans = cluster.KMeans(n_clusters=nbClusters, random_state=0).fit(P)
    linkUserCluster = kmeans.predict(P)
    
    #Get normalized centers and clusters
    (centersUsers,associatedUsers)=orderLabels(P,linkUserCluster)
    
    centersUsers = np.array(list(map(normalized,centersUsers)))
    
    mat += [[nbClusters,learningk(P,centersUsers,associatedUsers)]]
    

############################
### Clustering on Q
############################
  
#Get error depending on number of clusters
mat = [["nbClusters","error"]]

for nbClusters in range(1,101):
    
    print(nbClusters)

    # Clustering
    kmeans = cluster.KMeans(n_clusters=nbClusters, random_state=0).fit(Q)
    linkFilmCluster = kmeans.predict(Q)
    
    #Get normalized centers and clusters
    (centersFilms,associatedFilms)=orderLabels(Q,linkFilmCluster)
    
    centersFilms = np.array(list(map(normalized,centersFilms)))
    
    mat += [[nbClusters,learningk(Q,centersFilms,associatedFilms)]]
    


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


C:\Users\lamjo_000\Anaconda3\envs\python3\lib\site-packages\sklearn\decomposition\nmf.py:532: UserWarning: Iteration limit reached during fit. Solving for W exactly.
  warnings.warn("Iteration limit reached during fit. Solving for W exactly.")


### Comparison with metadata

In [33]:
#Figure 7 Figure 8

#Segmentation and interpretation of clusters with sklearn

##############################################
# Format metadata for users
##############################################

dfUser = pandas.read_csv(dfDirectory + "u.user", sep="\t")

#Create list with ages of users
ages=[24]

for i in range(0,8):
    ages.append(int(dfUser.iloc[i][0][2] + dfUser.iloc[i][0][3]))

for i in range(8,28):
    ages.append(int(dfUser.iloc[i][0][3] + dfUser.iloc[i][0][4]))

ages.append(7)

for i in range(29,98):
    ages.append(int(dfUser.iloc[i][0][3] + dfUser.iloc[i][0][4]))

for i in range(98,942):
    ages.append(int(dfUser.iloc[i][0][4] + dfUser.iloc[i][0][5]))

print("Get ages of users")
print(ages)

#Create list with gender of users
gender = [0]

for i in range(0,8):
    if dfUser.iloc[i][0][5] == 'M':
        gender.append(0)
    else:
        gender.append(1)

for i in range(8,28):
    if dfUser.iloc[i][0][6] == 'M':
        gender.append(0)
    else:
        gender.append(1)

gender.append(0)

for i in range(29,98):
    if dfUser.iloc[i][0][6] == 'M':
        gender.append(0)
    else:
        gender.append(1)

for i in range(98,942):
    if dfUser.iloc[i][0][7] == 'M':
        gender.append(0)
    else:
        gender.append(1)

print("Get genders of users")
print(gender)

#Create list with jobs of users
jobs = ['tec']

for i in range(0,8):
    jobs.append(dfUser.iloc[i][0][7] + dfUser.iloc[i][0][8] + dfUser.iloc[i][0][9])

for i in range(8,28):
    jobs.append(dfUser.iloc[i][0][8] + dfUser.iloc[i][0][9] + dfUser.iloc[i][0][10])

jobs.append('stu')

for i in range(29,98):
    jobs.append(dfUser.iloc[i][0][8] + dfUser.iloc[i][0][9] + dfUser.iloc[i][0][10])

for i in range(98,942):
    jobs.append(dfUser.iloc[i][0][9] + dfUser.iloc[i][0][10] + dfUser.iloc[i][0][11])

print("Get jobs of users")
print(jobs)




Get ages of users
[24, 53, 23, 24, 33, 42, 57, 36, 29, 53, 39, 28, 47, 45, 49, 21, 30, 35, 40, 42, 26, 25, 30, 21, 39, 49, 40, 32, 41, 7, 24, 28, 23, 38, 20, 19, 23, 28, 41, 38, 33, 30, 29, 26, 29, 27, 53, 45, 23, 21, 28, 18, 26, 22, 37, 25, 16, 27, 49, 50, 36, 27, 31, 32, 51, 23, 17, 19, 24, 27, 39, 48, 24, 39, 24, 20, 30, 26, 39, 34, 21, 50, 40, 32, 51, 26, 47, 49, 43, 60, 55, 32, 48, 26, 31, 25, 43, 49, 20, 36, 15, 38, 26, 27, 24, 61, 39, 44, 29, 19, 57, 30, 47, 27, 31, 40, 20, 21, 32, 47, 54, 32, 48, 34, 30, 28, 33, 24, 36, 20, 59, 24, 53, 31, 23, 51, 50, 46, 20, 30, 49, 13, 42, 53, 31, 45, 40, 33, 35, 20, 38, 33, 25, 25, 32, 25, 57, 50, 23, 27, 50, 25, 49, 47, 20, 47, 37, 48, 52, 53, 48, 55, 56, 30, 26, 28, 20, 26, 15, 22, 26, 36, 33, 37, 53, 39, 26, 42, 32, 30, 33, 42, 29, 38, 42, 49, 55, 21, 30, 40, 27, 41, 25, 52, 47, 14, 39, 43, 33, 39, 66, 49, 33, 26, 35, 22, 22, 37, 32, 30, 19, 29, 19, 31, 51, 28, 46, 21, 29, 28, 48, 45, 38, 60, 37, 44, 49, 42, 39, 23, 26, 33, 33, 28, 22, 19

In [34]:
import statistics

###############################################
# Interpretation of segmentation of users
###############################################

#Get caracteristics of each cluster
def caractCluster(i,dic):
    #i : cluster index
    #dic : dictionary {i : elements of cluster i}
    #Warning: here i corresponds to i+1 in database
    avg_age = 0
    median_age = 0
    var_age = 0
    rate_women = 0
    dic_occ = {'adm':0, 'art':0, 'doc':0, 'edu':0, 'eng':0, 'ent':0, 'exe':0, 'hea':0, 'hom':0, 'law':0,
    'lib':0, 'mar':0, 'non':0, 'oth':0, 'pro':0, 'ret':0, 'sal':0, 'sci':0, 'stu':0, 'tec':0, 'wri':0}
    ages_cluster = []
    occ_cluster = []
    gender_cluster = []
    for k in dic[i]:
        ages_cluster.append(ages[k])
        occ_cluster.append(jobs[k])
        gender_cluster.append(gender[k])
    avg_age = statistics.mean(ages_cluster)
    median_age = statistics.median(ages_cluster)
    if len(ages_cluster) != 1:
        var_age = statistics.variance(ages_cluster)
    rate_women = sum(gender_cluster)/float(len(gender_cluster))
    for k in occ_cluster:
        dic_occ[k] += 1
    dic_occ['adm'] = dic_occ['adm']/79
    dic_occ['art'] = dic_occ['art']/28
    dic_occ['doc'] = dic_occ['doc']/7
    dic_occ['edu'] = dic_occ['edu']/95
    dic_occ['eng'] = dic_occ['eng']/67
    dic_occ['ent'] = dic_occ['ent']/18
    dic_occ['exe'] = dic_occ['exe']/32
    dic_occ['hea'] = dic_occ['hea']/16
    dic_occ['hom'] = dic_occ['hom']/7
    dic_occ['law'] = dic_occ['law']/12
    dic_occ['lib'] = dic_occ['lib']/51
    dic_occ['mar'] = dic_occ['mar']/56
    dic_occ['non'] = dic_occ['non']/9
    dic_occ['oth'] = dic_occ['oth']/105
    dic_occ['pro'] = dic_occ['pro']/66
    dic_occ['ret'] = dic_occ['ret']/14
    dic_occ['sal'] = dic_occ['sal']/12
    dic_occ['sci'] = dic_occ['sci']/31
    dic_occ['stu'] = dic_occ['stu']/196
    dic_occ['tec'] = dic_occ['tec']/27
    dic_occ['wri'] = dic_occ['wri']/45
    return (dic_occ,max(dic_occ, key=dic_occ.get))


In [35]:
for i in range(nbClusters):
    print("cluster number")
    print(i)
    print("caracteristics of cluster")
    print(caractCluster(i,associatedUsers))

cluster number
0
caracteristics of cluster
({'oth': 0.01904761904761905, 'edu': 0.021052631578947368, 'eng': 0.0, 'hea': 0.0625, 'sal': 0.0, 'hom': 0.0, 'tec': 0.0, 'exe': 0.03125, 'non': 0.0, 'mar': 0.017857142857142856, 'ret': 0.0, 'adm': 0.0, 'art': 0.0, 'stu': 0.00510204081632653, 'lib': 0.0196078431372549, 'doc': 0.0, 'sci': 0.0, 'wri': 0.1111111111111111, 'law': 0.0, 'ent': 0.05555555555555555, 'pro': 0.045454545454545456}, 'wri')
cluster number
1
caracteristics of cluster
({'oth': 0.009523809523809525, 'edu': 0.010526315789473684, 'eng': 0.014925373134328358, 'hea': 0.0, 'sal': 0.0, 'hom': 0.0, 'tec': 0.0, 'exe': 0.0, 'non': 0.0, 'mar': 0.0, 'ret': 0.0, 'adm': 0.012658227848101266, 'art': 0.0, 'stu': 0.00510204081632653, 'lib': 0.0392156862745098, 'doc': 0.0, 'sci': 0.06451612903225806, 'wri': 0.022222222222222223, 'law': 0.0, 'ent': 0.0, 'pro': 0.015151515151515152}, 'sci')
cluster number
2
caracteristics of cluster
({'oth': 0.08571428571428572, 'edu': 0.07368421052631578, 'eng

In [36]:
# Scalar product between normalized vectors of users and films
def closest(centersFilms, centersUsers):
    #result = {}
    k_films = len(centersFilms)
    print("k_films")
    print(k_films)
    k_users = len(centersUsers)
    print("k_users")
    print(k_users)
    for i in range(k_users):
        a = np.array(centersUsers[i])/np.linalg.norm(np.array(centersUsers[i]))
        for j in range(0,k_films):
            b = np.array(centersFilms[j])/np.linalg.norm(np.array(centersFilms[j]))
    #Example :
    print(a @ b)

In [37]:
#Figure 9

#Segmentation and interpretation of clusters with sklearn


###########################################
# Attribute users to films
###########################################

closest(centersFilms, centersUsers)

k_films
100
k_users
100
0.293315662987


## Annexes

### Prediction with K-Means++

In our project, we use K-Means++ as a clustering method. But it is also possible to apply K-Means++ to prediction problems

In [48]:
#Find the coordinate j of the center corresponding to a point
def findCenter(usersClusters, centersClusters, user,j):
    val = usersClusters.values()
    nbClusters = len(centersClusters)
    indexCenter = 0
    for i in range(nbClusters):
        if any((x == user).all() for x in usersClusters[i]): #if user (which is an array) in users_clusters[i]:
            indexCenter = i
            break
    return centersClusters[indexCenter][j]

#Create dictionary with predicted ratings
def predictKMeans (rowMean,ratedTest,Matrix):
    dictionary = {}
    for i in ratedTest:
        for j in ratedTest[i]:
            bary = findCenter(usersClusters,centersClusters,Matrix[i],j)
            dictionary[(i,j)] =  roundDown(rowMean[i] + bary)
    return dictionary

In [49]:
steps=30
nbClusters=15

# Normalize matrix
ratings = replace0s(ratingsTrain, rowMean)

(usersClusters,centersClusters)=kmeansPP(ratings, nbClusters, steps)


# Create dictionary with predicted ratings
ratings = predictKMeans(rowMean,ratedTest,ratings)


#Calculate accuracy
print(calculateAccuracy(ratings,ratingsTestDic))   

0.639034955929


### Sparse Non Negative Matrix Factorization (SNMF)

In problems where dimension reduction is crucial, like image compressing, it is possible to restrict a matrix factor to its most important features only, using L1 norm penalization. (You want to use L0 norm, but can't for differentiability reasons. L1 is the closest and it works because it is not differentiable in 0).

Here we force sparseness onto the factor Q of films.

Refers to Algorithm 2 in article: http://www.jonathanleroux.org/pdf/LeRoux2015SparseNMF03.pdf

In [44]:
def update(A, P, Q, mu, r):
    PQ = P @ Q
    Q *= (P.T @ A) / (P.T @ PQ + mu)
    PQ = P @ Q
    oneNum = (P * (PQ @ Q.T)).sum(axis=0)
    oneDen = (P * (A @ Q.T)).sum(axis=0)
    P *= (A @ Q.T + P * np.matrix([oneDen for i in range(r)])) / (PQ @ Q.T + P * np.matrix([oneDen for i in range(r)]))
    row_sums = (P**2).sum(axis=0)**0.5
    P /= row_sums[np.newaxis,:]
    return P, Q


def SNMF(V, r, iterations=100, mu=0.1):
    n, m = V.shape
    P = np.random.random(n * r).reshape(n, r) * 4 + 1
    Q = np.random.random(r * m).reshape(r, m) * 4 + 1
    row_sums = (P**2).sum(axis=0)**0.5
    P /= row_sums[np.newaxis,:]
    for i in range(iterations):
        P, Q = update(A, P, Q, mu, r)
        PQ = P @ Q
        euclidean = np.nansum((A-PQ)**2)
    print("The euclidean distance is {d}".format(d=euclidean))
    return(P @ Q, P, Q)

In [50]:
SNMF(A,15)

The euclidean distance is 89005.38519427694


(array([[ 0.97361565,  0.9482587 ,  0.9760872 , ...,  0.92232144,
          0.91054249,  0.92801189],
        [ 1.07169906,  1.06991586,  1.07811814, ...,  0.9506623 ,
          0.95678686,  0.9630954 ],
        [ 0.97260784,  0.96954362,  0.96270912, ...,  1.00089893,
          0.99124403,  0.99680802],
        ..., 
        [ 1.28736925,  1.25381503,  1.27673459, ...,  4.93610052,
          4.88801815,  4.92428058],
        [ 1.19575742,  1.1603789 ,  1.18441069, ...,  4.98373545,
          4.94419653,  4.96918314],
        [ 1.20055212,  1.2200765 ,  1.19380339, ...,  4.97296964,
          4.94380512,  4.96757113]]),
 array([[ 0.01651331,  0.01821933,  0.0315125 , ...,  0.02048187,
          0.01419061,  0.01313434],
        [ 0.03126246,  0.01925697,  0.0398307 , ...,  0.04284268,
          0.02138491,  0.00861248],
        [ 0.03917808,  0.02307068,  0.03800563, ...,  0.00885042,
          0.02252872,  0.01472307],
        ..., 
        [ 0.01571074,  0.18032179,  0.05713984, ...,

### Weighted Sparse Non Negative Matrix Factorization (WSNMF) (Tentative name)

We apply SNMF on a matrix with missing data. In order to accomplish it, we combine the WNMF method with it.

In [51]:
def update(weightedA, P, Q, Weights, mu, r):
    weightedPQ = Weights * (P @ Q)
    Q *= (P.T @ weightedA) / (P.T @ weightedPQ + mu)
    weightedPQ = Weights * (P @ Q)
    oneNum = (P * (weightedPQ @ Q.T)).sum(axis=0)
    oneDen = (P * (weightedA @ Q.T)).sum(axis=0)
    P *= (weightedA @ Q.T + P * np.matrix([oneDen for i in range(r)])) / (weightedPQ @ Q.T + P * np.matrix([oneDen for i in range(r)]))
    row_sums = (P**2).sum(axis=0)**0.5
    P /= row_sums[np.newaxis,:]
    return P, Q


def WSNMF(A, r, iterations=100, mu=0.1):
    n, m = A.shape
    P = np.random.random(n * r).reshape(n, r) * 4 + 1
    Q = np.random.random(r * m).reshape(r, m) * 4 + 1
    Weights = A.copy()
    Weights[Weights != 0] = 1
    weightedA = Weights * A
    row_sums = (P**2).sum(axis=0)**0.5
    P /= row_sums[np.newaxis,:]
    for i in range(iterations):
        P, Q = update(weightedA, P, Q, Weights, mu, r)
        PQ = P @ Q
        weightedPQ = Weights * PQ
        euclidean = np.nansum((weightedA-weightedPQ)**2)
    print("The euclidean distance is {d}".format(d=euclidean))
    return(P @ Q, P, Q)

In [53]:
# Create new matrix
ratings,P,Q = WSNMF(ratingsTrain,9,1000)

# Create dictionary with predicted ratings
ratings = roundRatings(ratedTest,ratings,True)

# Calculate accuracy
print(calculateAccuracy(ratings,ratingsTestDic))

The euclidean distance is 44579.50412410784
0.809294871795
